In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [25]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [26]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [27]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('llp') 
# Actions that we try to detect
actions = np.array(['Are you a student','Are you hungry','Awesome','Best of luck','Call','Come','Do you understand','Hello','No','Please','Victory'])
# Thirty videos worth of data
no_sequences = 20
# Videos are going to be 30 frames in length
sequence_length = 30
# Folder start
start_folder = 20

In [28]:
for action in actions: 
    for sequence in range(0,no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [29]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

tb_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

actions = np.array(['Are you a student', 'Are you hungry', 'Awesome', 'Best of luck', 'Call', 'Come', 'Do you understand', 'Hello', 'No', 'Please', 'Victory'])
label_map = {label: num for num, label in enumerate(actions)}
DATA_PATH = os.path.join('llp')
sequence_length = 30
sequences, labels = [], []

for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Reshape the input data for 3D CNN
input_shape = (sequence_length, 126, 1, 1)  # Adjust input shape
X_train = X_train.reshape(-1, *input_shape)
X_test = X_test.reshape(-1, *input_shape)

model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(2, 2, 1)))
model.add(Conv3D(64, kernel_size=(3, 3, 1), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 1)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=150, callbacks=[tb_callback])


Epoch 1/150
6/6 [==============================] - 2s 166ms/step - loss: 2.3183 - categorical_accuracy: 0.1705
Epoch 2/150
6/6 [==============================] - 1s 139ms/step - loss: 2.0247 - categorical_accuracy: 0.2955
Epoch 3/150
6/6 [==============================] - 1s 134ms/step - loss: 1.7512 - categorical_accuracy: 0.3523
Epoch 4/150
6/6 [==============================] - 1s 136ms/step - loss: 1.5406 - categorical_accuracy: 0.3864
Epoch 5/150
6/6 [==============================] - 1s 147ms/step - loss: 1.4169 - categorical_accuracy: 0.4205
Epoch 6/150
6/6 [==============================] - 1s 139ms/step - loss: 1.2677 - categorical_accuracy: 0.5000
Epoch 7/150
6/6 [==============================] - 1s 140ms/step - loss: 1.1309 - categorical_accuracy: 0.5739
Epoch 8/150
6/6 [==============================] - 1s 136ms/step - loss: 0.9694 - categorical_accuracy: 0.6307
Epoch 9/150
6/6 [==============================] - 1s 134ms/step - loss: 0.8736 - categorical_accuracy: 0.6648
E

6/6 [==============================] - 1s 137ms/step - loss: 8.0452e-04 - categorical_accuracy: 1.0000
Epoch 147/150
6/6 [==============================] - 1s 142ms/step - loss: 7.7062e-04 - categorical_accuracy: 1.0000
Epoch 148/150
6/6 [==============================] - 1s 138ms/step - loss: 7.8073e-04 - categorical_accuracy: 1.0000
Epoch 149/150
6/6 [==============================] - 1s 149ms/step - loss: 7.3434e-04 - categorical_accuracy: 1.0000
Epoch 150/150
6/6 [==============================] - 1s 137ms/step - loss: 7.2487e-04 - categorical_accuracy: 1.0000


In [20]:
# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 124, 1, 32)    320       
                                                                 
 max_pooling3d (MaxPooling3  (None, 14, 62, 1, 32)     0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 60, 1, 64)     18496     
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 6, 30, 1, 64)      0         
 g3D)                                                            
                                                                 
 flatten (Flatten)           (None, 11520)             0         
                                                                 
 dense (Dense)               (None, 64)                7

In [33]:
res = model.predict(X_test)

2/2 [==============================] - 0s 12ms/step


In [34]:
actions[np.argmax(res[1])]

'No'

In [35]:
actions[np.argmax(y_test[1])]

'No'

In [36]:
model.save('3dcnntrail_1.h5')

/Applications/miniconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
from tensorflow.keras.models import load_model
# Load the saved model
model = load_model('3dcnntrail_1.h5')

In [38]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

2/2 [==============================] - 0s 28ms/step


array([[[39,  0],
        [ 0,  5]],

       [[42,  1],
        [ 0,  1]],

       [[40,  0],
        [ 0,  4]],

       [[35,  3],
        [ 0,  6]],

       [[39,  0],
        [ 3,  2]],

       [[39,  0],
        [ 0,  5]],

       [[39,  0],
        [ 0,  5]],

       [[40,  0],
        [ 0,  4]],

       [[42,  0],
        [ 0,  2]],

       [[39,  0],
        [ 1,  4]],

       [[42,  0],
        [ 0,  2]]])

In [4]:
accuracy_score(ytrue, yhat)

NameError: name 'accuracy_score' is not defined

In [40]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [5]:
from gtts import gTTS
import os
import pygame
from googletrans import Translator

def translate_text(text, src_lang, dest_lang):
    translator = Translator()
    
    # Translate text from source language to destination language
    translation1 = translator.translate(text, src=src_lang, dest=dest_lang)
    
    # Translate back from destination language to source language
    translation2 = translator.translate(translation1.text, src=dest_lang, dest=src_lang)
    
    return translation1.text, translation2.text


# Code for real-time prediction using MediaPipe
cap = cv2.VideoCapture(1)
sequence_length = 30
input_shape = (sequence_length, 126, 1, 1) 
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.8) as holistic:
    sequence = []
    sentence = []
    predictions = []
    threshold = 0.94
    vote_window_size = 10
    vote_window = []

    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make detections using MediaPipe
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            # Reshape the sequence for prediction
            sequence_for_prediction = np.expand_dims(sequence, axis=0)
            sequence_for_prediction = sequence_for_prediction.reshape(-1, *input_shape)
            res = model.predict(sequence_for_prediction)[0]
            action = actions[np.argmax(res)]

            # Add the action to the vote window
            vote_window.append(action)

            # Perform action recognition based on the majority vote
            if len(vote_window) >= vote_window_size:
                recognized_action = max(set(vote_window), key=vote_window.count)
                print(recognized_action)
                predictions.append(recognized_action)
                vote_window = []

                # Visualization logic
                if recognized_action == "_":
                    predictions.append("")
                else:
                    predictions.append(recognized_action)

                if np.unique(predictions[-10:])[0] == recognized_action:
                    if res[np.argmax(res)] > threshold:
                        if len(sentence) > 0:
                            if recognized_action != sentence[-1]:
                                sentence.append(recognized_action)
                                language = 'en'
                                myobj = gTTS(text=recognized_action, lang = language, slow = False)
                                translated_text1.save('welcome.mp3')
                                sound = pygame.mixer.Sound('welcome.mp3')
                                #Google Gemini concept
                                text = myobj
                                src_lang = 'en'
                                dest_lang = input("Enter destination language (e.g., 'fr' for French): ")

                                # Translate text bidirectionally
                                translated_text1, translated_text2 = translate_text(text, src_lang, dest_lang)

                                translated_text1.save('welcome_t.mp3')
                                sound = pygame.mixer.Sound('welcome_t.mp3')
                                sound.play()
                        else:
                            sentence.append(recognized_action)

                if len(sentence) > 3:
                    sentence = sentence[-3:]
        
        # Display sentence on the screen
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show the video feed with annotations
        cv2.imshow('OpenCV Feed', image)
        pygame.init()
        # def listToString(s): 
        #     # initialize an empty string
        #     str1 = ""         
        #     # traverse in the string
        #     for ele in sentence[-1:]:
        #         str1 += ele         
        #     # return string
        #     return str1
        # anss=listToString(sentence)
        # if anss!="":
            
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


NameError: name 'draw_styled_landmarks' is not defined